In [1]:
#본 내용은 kaggle Intermediate Machine Learning course(by Alexis Cook)를 기록해 놓은 것임을 밝힙니다. (https://www.kaggle.com/learn/intermediate-machine-learning)

In [2]:
from google.colab import drive 
drive.mount('/content/drive')
#구글 코랩환경 드라이브 연결

Mounted at /content/drive


# **Pipelines**

파이프라인은 데이터 사전 처리 및 모델링 코드를 정리하는 간단한 방법입니다. 특히 사전처리 및 모델링 단계를 번들로 묶어서 전체 번들을 한 단계처럼 사용할 수 있습니다.

1.   Cleaner Code: 각 단계에서 교육 및 검증 데이터를 수동적으로 추적할 필요가 없어집니다.
2.   Fewer Bugs: 단계를 잘못 사용하거나 전처리 단계를 잊어버리는 경우가 적어집니다.
3.   Easier to Productionize
4.   More Options for Model Validation: 교차검증(covers cross-validation)을 다루는 예



In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

data=pd.read_csv('/content/drive/MyDrive/ML-Study-Jam-in-DSC-Sookmyung/Intermediate Machine Learning/data/melb_data.csv')

y=data.Price
X=data.drop(['Price'],axis=1)

X_train_full, X_valid_full, y_train, y_valid=train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

categorical_cols=[cname for cname in X_train_full.columns if X_train_full[cname].nunique()<10 and X_train_full[cname].dtype == "object"]

numerical_cols=[cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

my_cols = categorical_cols + numerical_cols
X_train= X_train_full[my_cols].copy()
X_valid= X_valid_full[my_cols].copy()

In [7]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


**Define Preprocessing Steps**



*  imputes missing values in numerical data, and
imputes missing values 
*   and applies a one-hot encoding to categorical data.



In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
#numerical data 사전처리
numerical_transformer = SimpleImputer(strategy='constant')

#categorical data 사전처리
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

#numerical data and categorical data를 번들로 사전처리
preprocessor=ColumnTransformer(transformers=[('num',numerical_transformer, numerical_cols),
                                             ('cat',categorical_transformer, categorical_cols)])

**Define the Model**

In [9]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_estimators=100, random_state=0)

**Create and Evaluate the Pipeline**

Pipeline class를 사용하여 preprocessing and modeling steps의 bundles화 시킴

*   With the pipeline, we preprocess the training data and fit the model in a single line of code
*   With the pipeline, we supply the unprocessed features in X_valid to the predict() command, and the pipeline automatically preprocesses the features before generating predictions



In [11]:
from sklearn.metrics import  mean_absolute_error

my_pipeline=Pipeline(steps=[('preprocessor',preprocessor),('model', model)])

my_pipeline.fit(X_train, y_train)

preds=my_pipeline.predict(X_valid)

score = mean_absolute_error(y_valid,preds)
print('MAE:',score)

MAE: 160679.18917034855
